## Q1

In [45]:
# ----------------------------
# Part 1: 定义符号
# ----------------------------

from collections import defaultdict
import copy

class Term:
    def __init__(self, name):
        self.name = name

    def is_variable(self):
        # 变量以小写字母开头
        return self.name[0].islower()

    def __repr__(self):
        return self.name

    def __eq__(self, other):
        return isinstance(other, Term) and self.name == other.name

    def __hash__(self):
        return hash(self.name)

class Predicate:
    def __init__(self, name, args, is_negative=False):
        self.name = name
        self.args = args  # list of Term
        self.is_negative = is_negative

    def negate(self):
        return Predicate(self.name, self.args, not self.is_negative)

    def __repr__(self):
        sign = "¬" if self.is_negative else ""
        args_str = ", ".join([str(arg) for arg in self.args])
        return f"{sign}{self.name}({args_str})"

    def __eq__(self, other):
        return (self.name == other.name and
                self.args == other.args and
                self.is_negative == other.is_negative)

    def __hash__(self):
        return hash((self.name, tuple(self.args), self.is_negative))

class Clause:
    def __init__(self, predicates):
        self.predicates = predicates  # list of Predicate

    def __repr__(self):
        return " ∨ ".join([str(p) for p in self.predicates])

    def is_empty(self):
        return len(self.predicates) == 0

In [47]:
# ----------------------------
# Part 2: 定义规则
# ----------------------------

# 定义常量和变量
John = Term("John")
a = Term("a")  # 用于存在变量

# 定义变量
x = Term("x")
y = Term("y")
z = Term("z")

# 定义谓词
def Hound(term):
    return Predicate("Hound", [term])

def HowlsAtNight(term):
    return Predicate("HowlsAtNight", [term])

def Has(term1, term2):
    return Predicate("Has", [term1, term2])

def Cat(term):
    return Predicate("Cat", [term])

def Mouse(term):
    return Predicate("Mouse", [term])

def LightSleeper(term):
    return Predicate("LightSleeper", [term])

# 定义子句（规则）

# 1. ¬Hound(x) ∨ HowlsAtNight(x)
clause1 = Clause([Hound(x).negate(), HowlsAtNight(x)])

# 2a. ¬Cat(y) ∨ ¬Has(x, y)
clause2a = Clause([Cat(y).negate(), Has(x, y).negate()])

# 2b. ¬Mouse(z) ∨ ¬Has(x, z)
clause2b = Clause([Mouse(z).negate(), Has(x, z).negate()])

# 3. ¬LightSleeper(x) ∨ ¬HowlsAtNight(y) ∨ ¬Has(x, y)
clause3 = Clause([LightSleeper(x).negate(), HowlsAtNight(y).negate(), Has(x, y).negate()])

# 4a. Cat(y) ∨ Hound(y)
clause4a = Clause([Cat(y), Hound(y)])

# 4b. Has(John, y)
clause4b = Clause([Has(John, y)])

# 结论的否定
# 5a. LightSleeper(John)
clause5a = Clause([LightSleeper(John)])

# 5b. Mouse(a)
clause5b = Clause([Mouse(a)])

# 5c. Has(John, a)
clause5c = Clause([Has(John, a)])

# 收集所有子句
clauses = [clause1, clause2a, clause2b, clause3, clause4a, clause4b, clause5a, clause5b, clause5c]

In [51]:
def unify(x, y, subst):
    '''
    **description**
    合一算法，尝试合一两个项或谓词，并返回替换列表。

    **params**
    x: 第一个项或谓词。
    y: 第二个项或谓词。
    subst: 当前的替换列表。

    **returns**
    更新的替换列表，如果无法合一则返回 None。
    '''
    if subst is None:
        return None
    elif x == y:
        return subst
    elif isinstance(x, Term) and x.is_variable():
        return unify_var(x, y, subst)
    elif isinstance(y, Term) and y.is_variable():
        return unify_var(y, x, subst)
    elif isinstance(x, Predicate) and isinstance(y, Predicate):
        if x.name != y.name or len(x.args) != len(y.args):
            return None
        for a, b in zip(x.args, y.args):
            subst = unify(a, b, subst)
            if subst is None:
                return None
        return subst
    else:
        return None

def unify_var(var, x, subst):
    '''
    **description**
    合一变量。

    **params**
    var: 变量项。
    x: 另一个项。
    subst: 当前的替换列表。

    **returns**
    更新的替换列表，如果无法合一则返回 None。
    '''
    if var in subst:
        return unify(subst[var], x, subst)
    elif x in subst:
        return unify(var, subst[x], subst)
    else:
        if occurs_check(var, x, subst):
            return None
        subst[var] = x
        return subst

def occurs_check(var, x, subst):
    '''
    **description**
    检查变量是否出现在替换中，防止无限递归。

    **params**
    var: 变量项。
    x: 另一个项。
    subst: 当前的替换列表。

    **returns**
    如果出现返回 True，否则返回 False。
    '''
    if var == x:
        return True
    elif isinstance(x, Term) and x.is_variable() and x in subst:
        return occurs_check(var, subst[x], subst)
    return False

def substitute(predicate, subst):
    '''
    **description**
    在谓词中应用替换。

    **params**
    predicate: 需要替换的谓词。
    subst: 替换列表。

    **returns**
    应用替换后的新谓词。
    '''
    new_args = []
    for arg in predicate.args:
        if arg in subst:
            new_arg = subst[arg]
            while new_arg in subst:
                new_arg = subst[new_arg]
            new_args.append(new_arg)
        else:
            new_args.append(arg)
    return Predicate(predicate.name, new_args, predicate.is_negative)

def substitute_clause(clause, subst):
    '''
    **description**
    在子句中应用替换。

    **params**
    clause: 需要替换的子句。
    subst: 替换列表。

    **returns**
    应用替换后的新子句。
    '''
    new_predicates = []
    for pred in clause.predicates:
        new_pred = substitute(pred, subst)
        new_predicates.append(new_pred)
    return Clause(new_predicates)

# 归结算法
def resolution(clauses):
    '''
    **description**
    主归结算法，尝试在子句集合中证明矛盾（导出空子句）。

    **params**
    clauses: 初始子句集合。

    **returns**
    布尔值，表明是否成功证明矛盾。
    '''
    new = set()
    steps = 0  # 初始化推理步数计数器
    while True:
        n = len(clauses)
        pairs = [(clauses[i], clauses[j]) for i in range(n) for j in range(i + 1, n)]
        for (ci, cj) in pairs:
            steps += 1  # 每次归结操作计为一步
            # 推理步骤输出
            print(f"推理第{steps}步")
            print(f"输入句子: {ci} 和 {cj}")
            print(f"子句1: {ci}")
            print(f"子句2: {cj}")
            
            resolvents = resolve(ci, cj)
            
            for resolvent, unification_info, substitution_info in resolvents:
                # 打印 Unification 信息
                print("推理关系: Unification")
                print(f"{unification_info}")
                
                # 打印 Substitution 信息
                print("推理关系: Substitution")
                for sub_step in substitution_info:
                    print(f"{sub_step['new_clause']} = {sub_step['clause']}")
                    print(f"{sub_step['new_clause']} = {sub_step['clause']}")
                
                # 打印新句子
                print(f"新句子: {resolvent}")
                print("\n\n")  # 两个空行
                                
                if resolvent.is_empty():
                    print("导出空子句，证明成功！")
                    print(f"推理结束，总步数: {steps}")
                    return True
                new.add(resolvent)
        new_clauses = new - set(clauses)
        if not new_clauses:
            print("无法继续归结，证明失败。")
            print(f"推理结束，总步数: {steps}")
            return False
        clauses.extend(new_clauses)

def resolve(clause1, clause2):
    '''
    **description**
    尝试对两个子句进行归结，返回新的子句列表。

    **params**
    clause1: 第一个子句。
    clause2: 第二个子句。

    **returns**
    归结后的新子句列表，以及 Unification 和 Substitution 的信息。
    '''
    resolvents = []
    for pred1 in clause1.predicates:
        for pred2 in clause2.predicates:
            if pred1.name == pred2.name and pred1.is_negative != pred2.is_negative:
                subst = {}
                pred1_subst = copy.deepcopy(pred1)
                pred2_subst = copy.deepcopy(pred2)
                # 尝试合一
                print(f"尝试合一谓词 {pred1_subst} 和 {pred2_subst} 使用替换列表 {subst}")
                subst_result = unify(pred1_subst, pred2_subst, subst)
                if subst_result is not None:
                    unification_info = f"{pred1_subst} 和 {pred2_subst} 成功合一，替换列表: {subst_result}"
                    # 应用替换到剩余谓词
                    new_predicates = [substitute(p, subst_result) for p in clause1.predicates + clause2.predicates
                                     if p != pred1 and p != pred2]
                    new_clause = Clause(new_predicates)
                    
                    # 准备替换步骤信息
                    substitution_steps = []
                    for var, val in subst_result.items():
                        substitution_steps.append({
                            'new_clause': var.name if isinstance(var, Term) else var,
                            'clause': val.name if isinstance(val, Term) else val
                        })
                    
                    substitution_info = substitution_steps if substitution_steps else []
                    
                    resolvents.append((new_clause, unification_info, substitution_info))
                else:
                    unification_info = f"{pred1_subst} 和 {pred2_subst} 无法合一。"
                    substitution_info = []
                    # 即使合一失败，也可以选择记录或忽略
                    # 这里选择忽略失败的合一
    return [res for res in resolvents if res[0] is not None]

# 请在此处定义您的子句集合 `clauses`

# 开始推理
print("初始子句集合：")
for clause in clauses:
    print(clause)

print("\n开始归结推理...\n")

result = resolution(clauses)

初始子句集合：
¬Hound(x) ∨ HowlsAtNight(x)
¬Cat(y) ∨ ¬Has(x, y)
¬Mouse(z) ∨ ¬Has(x, z)
¬LightSleeper(x) ∨ ¬HowlsAtNight(y) ∨ ¬Has(x, y)
Cat(y) ∨ Hound(y)
Has(John, y)
LightSleeper(John)
Mouse(a)
Has(John, a)
¬Cat(a)
¬Has(x, a)
¬Hound(y) ∨ ¬LightSleeper(y) ∨ ¬Has(y, y)
HowlsAtNight(y) ∨ Cat(y)
¬Cat(y)
¬HowlsAtNight(y) ∨ ¬Has(John, y)
¬Mouse(y)
¬Has(x, y) ∨ Hound(y)
¬Mouse(a)
¬LightSleeper(John) ∨ ¬HowlsAtNight(y)
¬LightSleeper(John) ∨ ¬HowlsAtNight(a)

开始归结推理...

推理第1步
输入句子: ¬Hound(x) ∨ HowlsAtNight(x) 和 ¬Cat(y) ∨ ¬Has(x, y)
子句1: ¬Hound(x) ∨ HowlsAtNight(x)
子句2: ¬Cat(y) ∨ ¬Has(x, y)
推理第2步
输入句子: ¬Hound(x) ∨ HowlsAtNight(x) 和 ¬Mouse(z) ∨ ¬Has(x, z)
子句1: ¬Hound(x) ∨ HowlsAtNight(x)
子句2: ¬Mouse(z) ∨ ¬Has(x, z)
推理第3步
输入句子: ¬Hound(x) ∨ HowlsAtNight(x) 和 ¬LightSleeper(x) ∨ ¬HowlsAtNight(y) ∨ ¬Has(x, y)
子句1: ¬Hound(x) ∨ HowlsAtNight(x)
子句2: ¬LightSleeper(x) ∨ ¬HowlsAtNight(y) ∨ ¬Has(x, y)
尝试合一谓词 HowlsAtNight(x) 和 ¬HowlsAtNight(y) 使用替换列表 {}
推理关系: Unification
HowlsAtNight(x) 和 ¬HowlsAtNight(y) 成功合一，替换列表

## Q2

In [39]:
# ----------------------------
# Part 2: 定义规则
# ----------------------------


from collections import defaultdict
import copy

class Term:
    def __init__(self, name):
        """
        初始化一个项（变量或常量）。

        :param name: 项的名称
        """
        self.name = name;

    def isVariable(self):
        """
        判断项是否为变量（名称以小写字母开头）。

        :return: 如果是变量返回 True，否则返回 False
        """
        return self.name[0].islower();

    def __repr__(self):
        return self.name;

    def __eq__(self, other):
        return isinstance(other, Term) and self.name == other.name;

    def __hash__(self):
        return hash(self.name);

class Predicate:
    def __init__(self, name, args, isNegative=False):
        """
        初始化一个谓词。

        :param name: 谓词名称
        :param args: 参数列表（Term 对象列表）
        :param isNegative: 是否为否定谓词
        """
        self.name = name;
        self.args = args;  # list of Term
        self.isNegative = isNegative;

    def negate(self):
        """
        返回该谓词的否定形式。

        :return: 否定的 Predicate 对象
        """
        return Predicate(self.name, self.args, not self.isNegative);

    def __repr__(self):
        sign = "¬" if self.isNegative else "";
        argsStr = ", ".join([str(arg) for arg in self.args]);
        return f"{sign}{self.name}({argsStr})";

    def __eq__(self, other):
        return (self.name == other.name and
                self.args == other.args and
                self.isNegative == other.isNegative);

    def __hash__(self):
        return hash((self.name, tuple(self.args), self.isNegative));

class Clause:
    def __init__(self, predicates):
        """
        初始化一个子句（由谓词组成的析取）。

        :param predicates: 谓词列表
        """
        self.predicates = predicates;  # list of Predicate

    def __repr__(self):
        return " ∨ ".join([str(p) for p in self.predicates]);

    def isEmpty(self):
        """
        判断子句是否为空（没有任何谓词）。

        :return: 如果为空返回 True，否则返回 False
        """
        return len(self.predicates) == 0;


In [41]:
# ----------------------------
# Part 2: 定义符号
# ----------------------------


# 定义常量和变量
x = Term("x");
y = Term("y");

# 定义常量个体
a = Term("a");  # 表示进入国家的毒贩，只被毒贩搜查
b = Term("b");  # 表示既是海关官员又是毒贩的个体

# 定义谓词
def entered(term):
    return Predicate("entered", [term]);

def VIP(term):
    return Predicate("VIP", [term]);

def searchedByCustoms(term):
    return Predicate("searchedByCustoms", [term]);

def drugDealer(term):
    return Predicate("drugDealer", [term]);

def searchedByDrugDealer(term):
    return Predicate("searchedByDrugDealer", [term]);

def customsOfficial(term):
    return Predicate("customsOfficial", [term]);

# 定义子句（规则）

# 1. ¬entered(x) ∨ VIP(x) ∨ searchedByCustoms(x)
clause1 = Clause([entered(x).negate(), VIP(x), searchedByCustoms(x)]);

# 2a. drugDealer(a)
clause2a = Clause([drugDealer(a)]);

# 2b. entered(a)
clause2b = Clause([entered(a)]);

# 2c. searchedByDrugDealer(a)
clause2c = Clause([searchedByDrugDealer(a)]);

# 2d. ¬searchedByCustoms(a)
clause2d = Clause([searchedByCustoms(a).negate()]);

# 3. ¬drugDealer(x) ∨ ¬VIP(x)
clause3 = Clause([drugDealer(x).negate(), VIP(x).negate()]);

# 4a. customsOfficial(b)
clause4a = Clause([customsOfficial(b)]);

# 4b. drugDealer(b)
clause4b = Clause([drugDealer(b)]);

# 收集所有子句
clauses = [clause1, clause2a, clause2b, clause2c, clause2d, clause3, clause4a, clause4b];


In [30]:
# ----------------------------
# Part 3: 推理机制
# ----------------------------

def unify(x, y, subst):
    '''
    **description**
    合一算法，尝试合一两个项或谓词，并返回替换列表。

    **params**
    x: 第一个项或谓词。
    y: 第二个项或谓词。
    subst: 当前的替换列表。

    **returns**
    更新的替换列表，如果无法合一则返回 None。
    '''
    if subst is None:
        return None;
    elif x == y:
        return subst;
    elif isinstance(x, Term) and x.isVariable():
        return unifyVar(x, y, subst);
    elif isinstance(y, Term) and y.isVariable():
        return unifyVar(y, x, subst);
    elif isinstance(x, Predicate) and isinstance(y, Predicate):
        if x.name != y.name or len(x.args) != len(y.args):
            return None;
        for a, b in zip(x.args, y.args):
            subst = unify(a, b, subst);
            if subst is None:
                return None;
        return subst;
    else:
        return None;

def unifyVar(var, x, subst):
    '''
    **description**
    合一变量。

    **params**
    var: 变量项。
    x: 另一个项。
    subst: 当前的替换列表。

    **returns**
    更新的替换列表，如果无法合一则返回 None。
    '''
    if var in subst:
        return unify(subst[var], x, subst);
    elif x in subst:
        return unify(var, subst[x], subst);
    else:
        if occursCheck(var, x, subst):
            return None;
        subst[var] = x;
        return subst;

def occursCheck(var, x, subst):
    '''
    **description**
    检查变量是否出现在替换中，防止无限递归。

    **params**
    var: 变量项。
    x: 另一个项。
    subst: 当前的替换列表。

    **returns**
    如果出现返回 True，否则返回 False。
    '''
    if var == x:
        return True;
    elif isinstance(x, Term) and x.isVariable() and x in subst:
        return occursCheck(var, subst[x], subst);
    return False;

def substitute(predicate, subst):
    '''
    **description**
    在谓词中应用替换。

    **params**
    predicate: 需要替换的谓词。
    subst: 替换列表。

    **returns**
    应用替换后的新谓词。
    '''
    newArgs = [];
    for arg in predicate.args:
        if arg in subst:
            newArg = subst[arg];
            while newArg in subst:
                newArg = subst[newArg];
            newArgs.append(newArg);
        else:
            newArgs.append(arg);
    return Predicate(predicate.name, newArgs, predicate.isNegative);

def substituteClause(clause, subst):
    '''
    **description**
    在子句中应用替换。

    **params**
    clause: 需要替换的子句。
    subst: 替换列表。

    **returns**
    应用替换后的新子句。
    '''
    newPredicates = [];
    for pred in clause.predicates:
        newPred = substitute(pred, subst);
        newPredicates.append(newPred);
    return Clause(newPredicates);

def resolution(clauses):
    '''
    **description**
    主归结算法，尝试在子句集合中证明矛盾（导出空子句）。

    **params**
    clauses: 初始子句集合。

    **returns**
    布尔值，表明是否成功证明矛盾。
    '''
    new = set();
    steps = 0;  # 初始化推理步数计数器
    while True:
        n = len(clauses);
        pairs = [(clauses[i], clauses[j]) for i in range(n) for j in range(i + 1, n)];
        for (ci, cj) in pairs:
            steps += 1;  # 每次归结操作计为一步
            print(f"推理第{steps}步");
            print(f"输入句子: {ci} 和 {cj}");
            print(f"子句1: {ci}");
            print(f"子句2: {cj}");
            
            resolvents = resolve(ci, cj);
            
            for resolvent, unification_info, substitution_info in resolvents:
                print("推理关系: Unification");
                print(f"统一信息: {unification_info}");
                print("推理关系: Substitution");
                print(f"替换列表: {substitution_info}");
                print(f"新句子: {resolvent}");
                
                if substitution_info:
                    for sub_step in substitution_info:
                        print(f"{sub_step['new_clause']} = {sub_step['clause']}");
                        print(f"{sub_step['new_clause']} = {sub_step['clause']}");
                
                print("\n");
                                
                if resolvent.isEmpty():
                    print("导出空子句，证明成功！");
                    print(f"推理结束，总步数: {steps}");
                    return True;
                new.add(resolvent);
        newClauses = new - set(clauses);
        if not newClauses:
            print("无法继续归结，证明失败。");
            print(f"推理结束，总步数: {steps}");
            return False;
        clauses.extend(newClauses);

def resolve(clause1, clause2):
    '''
    **description**
    尝试对两个子句进行归结，返回新的子句列表。

    **params**
    clause1: 第一个子句。
    clause2: 第二个子句。

    **returns**
    归结后的新子句列表，以及 Unification 和 Substitution 的信息。
    '''
    resolvents = [];
    for pred1 in clause1.predicates:
        for pred2 in clause2.predicates:
            if pred1.name == pred2.name and pred1.isNegative != pred2.isNegative:
                subst = {};
                pred1Subst = copy.deepcopy(pred1);
                pred2Subst = copy.deepcopy(pred2);
                unification_before = f"{pred1Subst} 和 {pred2Subst}";
                subst_result = unify(pred1Subst, pred2Subst, subst);
                if subst_result is not None:
                    unification_info = f"{pred1Subst} 和 {pred2Subst} 成功合一，替换列表: {subst_result}";
                    newPredicates = [substitute(p, subst_result) for p in clause1.predicates + clause2.predicates
                                     if p != pred1 and p != pred2];
                    newClause = Clause(newPredicates);
                    
                    substitution_steps = [];
                    for var, val in subst_result.items():
                        substitution_steps.append({
                            'new_clause': var,
                            'clause': val
                        });
                    
                    substitution_info = substitution_steps if substitution_steps else None;
                    
                    resolvents.append((newClause, unification_info, substitution_info));
                else:
                    unification_info = f"{pred1Subst} 和 {pred2Subst} 无法合一。";
                    substitution_info = None;
                    resolvents.append((None, unification_info, substitution_info));
    return [res for res in resolvents if res[0] is not None];


print("初始子句集合：");
for clause in clauses:
    print(clause);

print("\n开始归结推理...\n");

result = resolution(clauses);


初始子句集合：
¬entered(x) ∨ VIP(x) ∨ searchedByCustoms(x)
drugDealer(a)
entered(a)
searchedByDrugDealer(a)
¬searchedByCustoms(a)
¬drugDealer(x) ∨ ¬VIP(x)
customsOfficial(b)
drugDealer(b)
VIP(a) ∨ searchedByCustoms(a)
¬VIP(b)
¬VIP(x)
¬entered(x) ∨ searchedByCustoms(x) ∨ ¬drugDealer(x)
¬entered(a) ∨ VIP(a)
¬drugDealer(a) ∨ searchedByCustoms(a)
VIP(a)
searchedByCustoms(x)
¬entered(x) ∨ searchedByCustoms(x)
¬VIP(x)
¬entered(a) ∨ VIP(a)
VIP(a) ∨ searchedByCustoms(a)
¬entered(b) ∨ searchedByCustoms(b)
¬entered(x) ∨ searchedByCustoms(x) ∨ ¬drugDealer(x)
searchedByCustoms(x) ∨ ¬drugDealer(x)
¬entered(a)
¬drugDealer(a) ∨ ¬entered(a)
¬entered(x) ∨ searchedByCustoms(x)
¬entered(a)
¬entered(x) ∨ ¬drugDealer(x)
¬entered(x) ∨ searchedByCustoms(x)
VIP(a)
¬VIP(b)
searchedByCustoms(b)

开始归结推理...

推理第1步
输入句子: ¬entered(x) ∨ VIP(x) ∨ searchedByCustoms(x) 和 drugDealer(a)
子句1: ¬entered(x) ∨ VIP(x) ∨ searchedByCustoms(x)
子句2: drugDealer(a)
推理第2步
输入句子: ¬entered(x) ∨ VIP(x) ∨ searchedByCustoms(x) 和 entered(a)
子句1: ¬